In [42]:
from selenium import webdriver
import time, os
import io
import requests
from bs4 import BeautifulSoup
from PIL import Image
import hashlib
import re

from selenium.webdriver.chrome.options import Options

In [10]:
# When migrating away from Jupyter notebook, this will proabbly be moved to a __main__
options = Options()
options.headless = False

DRIVER_PATH = "./chromedriver"
wd = webdriver.Chrome(executable_path=DRIVER_PATH, options=options)

In [145]:
# search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
# query = "hellfire"
# wd.get(search_url.format(q=query))

In [59]:
blocked_sites = ["apartments.com", "bizquest.com", "chicagoheightslaundromat.com", "loopnet.com"]

In [47]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)
        
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
    
    wd.get(search_url.format(q=query))
    
    image_urls = set()
    image_count = 0
    results_start = 0
    
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)
        
        thumbnail_results = wd.find_elements_by_css_selector("img.rg_ic")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue
                
            actual_images = wd.find_elements_by_css_selector('img.irc_mi')
            for actual_image in actual_images:
                if actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))
                    
            # Filter out sites on the block list
            # Considering moving this check functionality to the above loop
            unblocked_urls = set()
            for url in image_urls:
                for b in blocked_sites:
                    if re.search(b, url) is None:
                        unblocked_urls.add(url)
                    
#             image_count = len(image_urls)
            image_count = len(unblocked_urls)
            
            if image_count >= max_links_to_fetch:
                print(f"Found: {image_count} image links, done!")
                break
        else:
            print("Found:", image_count, "image links, looking for more...")
            time.sleep(5)
            load_more_button = wd.find_element_by_css_selector(".ksb")
            if load_more_button:
                wd.execute_script("document.querySelector('.ksb').click();")

        results_start = len(thumbnail_results)
#     return image_urls
    return unblocked_urls

In [19]:
fetch_image_urls("ferret", 5, wd)

Found: 200 search results. Extracting links from 0:200
Found: 5 image links, done!


{'https://media.daysoftheyear.com/20171223130951/ferret-day-1920x580.jpg',
 'https://petco.scene7.com/is/image/PETCO/153044-right-4?$ProductDetail-large$',
 'https://s7d1.scene7.com/is/image/PETCO/153044-left-1?$ProductDetail-large$',
 'https://upload.wikimedia.org/wikipedia/commons/3/32/Ferret_2008.png',
 'https://www.cdc.gov/healthypets/images/pets/ferret-on-white-background.jpg'}

In [7]:
wd.quit()

In [4]:
def persist_image(folder_path:str, url:str):
    try:
        image_content = requests.get(url).content
    except Exception as e:
        print(f"ERROR - could not download {url} - {e}")
        
    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path, hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [29]:
def search_and_download(search_term:str, driver_path:str, target_path='./data/images', number_images=5, headless=False):
    search_term_parse = '_'.join(search_term.lower().split(' '))
    target_folder = os.path.join(target_path, search_term_parse)
    
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    
    options = Options()
    options.headless = headless
    with webdriver.Chrome(executable_path=driver_path, options=options) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    with open(target_folder + '/' + search_term_parse + '.csv', 'w') as urls_file:
        for r in res:
            urls_file.write(r + "\n")
    urls_file.close()
            
    for elem in res:
        persist_image(target_folder, elem)

In [28]:
search_term = "hell and damnation"
DRIVER_PATH = "./chromedriver"

search_and_download(search_term=search_term, driver_path=DRIVER_PATH)

hell_and_damnation
Found: 100 search results. Extracting links from 0:100
Found: 5 image links, done!
SUCCESS - saved https://1.bp.blogspot.com/-xQtY-NgFhtA/XLdLqGUpe2I/AAAAAAAADdg/K2KYL20X7bU9rsK4ItoGcHB_YM1C136FwCLcBGAs/s1600/Painkiller-Hell-and-Damnation-wallpaper-1024x640.jpg - as ./data/images/hell_and_damnation/fd49afba28.jpg
SUCCESS - saved https://i.ytimg.com/vi/Rz-I0DYk0TI/maxresdefault.jpg - as ./data/images/hell_and_damnation/24d892ed1c.jpg
SUCCESS - saved https://i.ytimg.com/vi/KsuOxD8Vd2E/maxresdefault.jpg - as ./data/images/hell_and_damnation/f5f66d6abe.jpg
SUCCESS - saved http://www.gamesreviews.com/wp-content/uploads/2013/07/Painkiller2_690x388.jpg - as ./data/images/hell_and_damnation/a2f235b557.jpg
SUCCESS - saved https://steamcdn-a.akamaihd.net/steam/apps/214870/ss_9d721e472d69a6b8ec08e3f49e7656b801ce321c.1920x1080.jpg?t=1562844668 - as ./data/images/hell_and_damnation/3d75f513c7.jpg


In [49]:
empties = ["empty basement", "empty laundry room", "unfinished basement"]
people = ["person", "full body man", "woman", "full body woman", "person sitting", "person crouching", "person bending"]
laundry = ["doing laundry"]

In [55]:
laundromats = ["empty laundromat", "busy laundromat"]
laundromats = ["busy laundromat"]

In [58]:
for term in laundromats:
    search_and_download(term, driver_path=DRIVER_PATH, number_images=300)

busy_laundromat
Found: 200 search results. Extracting links from 0:200
Found: 198 image links, looking for more...
Found: 300 search results. Extracting links from 200:300
Found: 298 image links, looking for more...
Found: 500 search results. Extracting links from 300:500
Found: 300 image links, done!
SUCCESS - saved https://images.bizbuysell.com/shared/listings/161/1614914/c6bd1873-beea-41fa-a717-12caf5e57830-W768.png - as ./data/images/busy_laundromat/6a47ae9b4b.jpg
SUCCESS - saved https://www.tracysnewyorklife.com/wp-content/uploads/2015/03/laundromatflickr.jpg - as ./data/images/busy_laundromat/3b4a6eee39.jpg
SUCCESS - saved https://www.sueslaundromat.com/images/Warrenton-coin-operated-laundromat.jpg - as ./data/images/busy_laundromat/ba2f450833.jpg
ERROR - Could not save https://www.sunnysudslaundromat.com/wp-content/uploads/2017/02/washfold2.jpg - cannot identify image file <_io.BytesIO object at 0x7faef5304c50>
SUCCESS - saved https://www.cgilaundry.com/images/vended/storetours/

SUCCESS - saved https://cdn.businessyab.com/assets/uploads/1aa8cb52f969718329c002c4ea181d6f__united_states_new_york_nassau_county_hempstead_south_franklin_street_495_busy_bee_laundromat_38947.jpg - as ./data/images/busy_laundromat/94e241c905.jpg
SUCCESS - saved https://pbs.twimg.com/profile_images/1083025602255507458/jIKFUGRu_400x400.jpg - as ./data/images/busy_laundromat/001c28a6f2.jpg
SUCCESS - saved https://cdn.website.thryv.com/821db0fbca184ffbba17940bdb988105/dms3rep/multi/mobile/1555879-GettyImages-871387534.jpg - as ./data/images/busy_laundromat/4be99eed94.jpg
SUCCESS - saved https://laundromats-for-sale.com/wp-content/uploads/2019/07/Screen-Shot-2019-07-18-at-10.39.14-AM.png - as ./data/images/busy_laundromat/addaa7f1ab.jpg
SUCCESS - saved https://cdn.businessyab.com/assets/uploads/a14703237b5de7716324721bf7a564ad__united_states_new_york_nassau_county_hempstead_south_franklin_street_495_busy_bee_laundromat_38947.jpg - as ./data/images/busy_laundromat/8d4bff79f3.jpg
ERROR - coul

KeyboardInterrupt: 

In [ ]:
for term in empties:
    search_and_download(term, driver_path=DRIVER_PATH, number_images=300)

In [50]:
# Code hung up on "unfinished basement"
# Adding a block list for a URL; the system is hung up on one site.
search_and_download(empties[2], driver_path=DRIVER_PATH, number_images=300)

unfinished_basement
Found: 200 search results. Extracting links from 0:200
Found: 196 image links, looking for more...
Found: 300 search results. Extracting links from 200:300
Found: 296 image links, looking for more...
Found: 400 search results. Extracting links from 300:400
Found: 300 image links, done!
SUCCESS - saved http://www.unemamancreative.com/wp-content/uploads/2015/05/unfinished-basement-wall-ideas.jpg - as ./data/images/unfinished_basement/b09500aaf2.jpg
SUCCESS - saved https://sehatwae.com/wp-content/uploads/2018/05/modern-minimalist-unfinished-basement-ideas.jpg - as ./data/images/unfinished_basement/52ff137d52.jpg
SUCCESS - saved https://demiandashton.org/wp-content/uploads/2018/04/basement_ideas_for_unfinished_basements.jpg - as ./data/images/unfinished_basement/d19dd3ae4e.jpg
SUCCESS - saved https://static.wixstatic.com/media/c0a6c9_897d2b3e580b40298bd2adb925dbbd79~mv2.jpg/v1/fill/w_240,h_320,al_c,q_90/file.jpg - as ./data/images/unfinished_basement/40f1ba0502.jpg
ERRO

SUCCESS - saved https://s3.amazonaws.com/treehouse-content/uploads/photo_gallery/medium/90938-schulze-after3.jpg?v=1465823357 - as ./data/images/unfinished_basement/927f51d331.jpg
SUCCESS - saved https://s3.amazonaws.com/treehouse-content/uploads/before_after/1207/medium/5c475ab8ae1d6_20181030040843.jpg - as ./data/images/unfinished_basement/19144c3715.jpg
SUCCESS - saved https://s3.amazonaws.com/treehouse-content/uploads/photo_gallery/large/45436-img0005.jpg - as ./data/images/unfinished_basement/ab2d61b7b3.jpg
SUCCESS - saved https://i2.wp.com/www.smithandsonsmd.com/wp-content/uploads/2019/05/ceiling-chairs-clean-1571459.jpg?resize=300%2C196&ssl=1 - as ./data/images/unfinished_basement/e41169b856.jpg
SUCCESS - saved https://i0.wp.com/zuvoria.com/wp-content/uploads/2019/03/Throw-Area-Rugs-Into-the-Floor.png?resize=668%2C500&ssl=1 - as ./data/images/unfinished_basement/20a289c058.jpg
SUCCESS - saved https://images.squarespace-cdn.com/content/v1/552c0c04e4b071ae9227aa38/1429121121537-BQ

SUCCESS - saved https://porch.com/advice/wp-content/uploads/2014/03/8c0f5f293187f4fce035ac1c4095a6bd-1-960x5001.jpg - as ./data/images/unfinished_basement/fa569352a2.jpg
SUCCESS - saved https://i.pinimg.com/originals/06/21/a8/0621a830c7963022c7236770a96bee6d.jpg - as ./data/images/unfinished_basement/6578b0f22d.jpg
SUCCESS - saved https://s3-production.bobvila.com/slides/21389/widened/painted_pipes_and_beams.jpg?1501003283 - as ./data/images/unfinished_basement/f9a3f54c60.jpg
SUCCESS - saved https://neonsanctum.com/wp-content/uploads/cute-painting-unfinished-basement-ceiling-ideas_100860.jpg - as ./data/images/unfinished_basement/e7e2b8b0b8.jpg
SUCCESS - saved http://kundehomes.com/wp-content/gallery/57203-e-taggart-pr-se-basement/04-IMG_9823.JPG - as ./data/images/unfinished_basement/467f081769.jpg
SUCCESS - saved https://media.istockphoto.com/photos/unfinished-basement-picture-id1091887602 - as ./data/images/unfinished_basement/a73fd1b84c.jpg
ERROR - could not download http://www.the

SUCCESS - saved https://s3.amazonaws.com/treehouse-content/uploads/photo_gallery/large/13542-an-unfinished-basement-with-no-storage-space-at-a-home-in-bridgeport-ct.jpg - as ./data/images/unfinished_basement/669f48bd26.jpg
ERROR - could not download http://www.theweatheredfox.com/wp-content/uploads/2017/05/Unfinished-basement-ideas-to-sell-a-house-16.jpg - HTTPSConnectionPool(host='www.theweatheredfox.com', port=443): Max retries exceeded with url: /wp-content/uploads/2017/05/Unfinished-basement-ideas-to-sell-a-house-16.jpg (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))
ERROR - Could not save http://www.theweatheredfox.com/wp-content/uploads/2017/05/Unfinished-basement-ideas-to-sell-a-house-16.jpg - local variable 'image_content' referenced before assignment
SUCCESS - saved https://cmongetcrafty.com/wp-content/uploads/2015/05/Cmon-Get-Crafty-Basement-Playroom-4b-700x394.jpg - as ./data/images/unf

SUCCESS - saved https://www.semiglossdesign.com/wp-content/uploads/2019/11/unfinished-basement-ceiling.jpg - as ./data/images/unfinished_basement/05437540ab.jpg
SUCCESS - saved http://www.time2remodel.com/wp-content/uploads/2016/02/from-storage-to-family-fun.jpg - as ./data/images/unfinished_basement/5c0f7afe26.jpg
ERROR - Could not save https://www.hardrockconcretecoatings.com/wp-content/uploads/2017/02/5-ways-to-fix-your-unfinished-basement.jpeg - cannot identify image file <_io.BytesIO object at 0x7faef52379b0>
SUCCESS - saved https://www.constructionprotips.com/wp-content/uploads/2017/12/framingagainstbasement1217-1024x1024.jpg - as ./data/images/unfinished_basement/41be462ce9.jpg
SUCCESS - saved https://www.boulderarchitects.com/wp-content/uploads/2013/09/9519695644_0b09f870d8_z.jpg - as ./data/images/unfinished_basement/143c081144.jpg
SUCCESS - saved http://instantcoelho.com/public/Unfinished-Basement-Ideas-You-Can-Look-Basement-Design.jpg - as ./data/images/unfinished_basement/f

SUCCESS - saved https://assets.leaffiltercdn.com/app/uploads/2017/02/unfinished-basement-ideas-2nd-image.jpg - as ./data/images/unfinished_basement/3136f36da7.jpg
SUCCESS - saved https://s3.amazonaws.com/treehouse-content/uploads/photo_gallery/large/80370-teeple-before1.jpg - as ./data/images/unfinished_basement/89a0d91a0f.jpg
SUCCESS - saved https://c8.alamy.com/comp/KE6HA6/unfinished-basement-interior-of-middle-class-american-home-in-kentucky-KE6HA6.jpg - as ./data/images/unfinished_basement/b39cc91611.jpg
SUCCESS - saved http://hanoveravenue.com/wp-content/uploads/2015/03/basement-as-a-loft-sink-view-1024x486.jpg - as ./data/images/unfinished_basement/cfda646c1a.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/doorways-unfinished-basement-abandoned-residental-600w-1220101057.jpg - as ./data/images/unfinished_basement/d376511ab0.jpg
SUCCESS - saved https://i.ytimg.com/vi/RIzNQhfVFWA/maxresdefault.jpg - as ./data/images/unfinished_basement/76f7389ce4.jpg
SUCCESS - saved 

SUCCESS - saved https://s3.amazonaws.com/treehouse-content/uploads/photo_gallery/large/54392-leebefore1.jpg - as ./data/images/unfinished_basement/b448fe1537.jpg
SUCCESS - saved https://postmediaottawacitizen2.files.wordpress.com/2018/03/before-2-jpg.jpg?w=371&quality=65&strip=all - as ./data/images/unfinished_basement/9727d7346e.jpg
SUCCESS - saved https://www.budgetdumpster.com/blog/wp-content/uploads/2018/06/basement-organization-tips.jpg - as ./data/images/unfinished_basement/674d2e7f3e.jpg
SUCCESS - saved https://st.hzcdn.com/simgs/a3e1b70e007ec0a5_8-7504/industrial-basement.jpg - as ./data/images/unfinished_basement/559ee0eece.jpg
SUCCESS - saved https://twistedsifter.files.wordpress.com/2014/12/animated-timelapse-finishing-basement-in-60-seconds.jpg - as ./data/images/unfinished_basement/c421e3472e.jpg
SUCCESS - saved http://www.jimmyssliceventuracrafteats.com/wp-content/uploads/2015/07/unfinished-basement-playroom-ideas.jpg - as ./data/images/unfinished_basement/89d1c5d09a.jpg


In [34]:
for term in people:
    search_and_download(term, driver_path=DRIVER_PATH, number_images=100)

person
Found: 100 search results. Extracting links from 0:100
Found: 98 image links, looking for more...
Found: 300 search results. Extracting links from 100:300
Found: 100 image links, done!
SUCCESS - saved https://www.theglobeandmail.com/resizer/ZTY0dRYn2x4dgSkwSPG-nyIjflU=/620x0/filters:quality(80)/arc-anglerfish-tgam-prod-tgam.s3.amazonaws.com/public/5HSZVXDII5BRRHH4S6KE4WZ7RE.jpg - as ./data/images/person/eb566cfc36.jpg
SUCCESS - saved https://i.insider.com/5df28947fd9db27324080cb2?width=1100&format=jpeg&auto=webp - as ./data/images/person/3e384a63a7.jpg
SUCCESS - saved https://a57.foxnews.com/media2.foxnews.com/BrightCove/694940094001/2019/12/11/931/524/694940094001_6114905482001_6114907573001-vs.jpg?ve=1&tl=1 - as ./data/images/person/0f3f2eadcb.jpg
SUCCESS - saved https://cdn1.thr.com/sites/default/files/imagecache/NFE_Landscape/2019/12/gigi_gorgeous_240_bw_1-h_2019.jpg - as ./data/images/person/a39e240dab.jpg
SUCCESS - saved https://www.lifewire.com/thmb/JbIIh6ewCm49Eon66Exjbx

SUCCESS - saved https://cdn.psychologytoday.com/sites/default/files/styles/article-inline-half-caption/public/field_blog_entry_images/2018-09/shutterstock_648907024.jpg?itok=0hb44OrI - as ./data/images/person/12879b24e3.jpg
ERROR - Could not save https://www.success.com/wp-content/uploads/2018/08/TheArtofTrueConfidence-1024x682.jpg - cannot identify image file <_io.BytesIO object at 0x7faef5314710>
SUCCESS - saved https://metabolicdevices.com/wp-content/uploads/2017/04/person-4.jpg - as ./data/images/person/714d139d70.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/a/a0/Pierre-Person.jpg - as ./data/images/person/f31fe0f05f.jpg
SUCCESS - saved https://www.incimages.com/uploaded_files/image/970x450/getty_504134147_174741.jpg - as ./data/images/person/c95be63a9e.jpg
SUCCESS - saved https://hope1032.com.au/wp-content/uploads/2019/10/Waiting-for-friends-1200-x-480.png - as ./data/images/person/1f7d91d5af.jpg
SUCCESS - saved https://tvguide1.cbsistatic.com/i/2018/02/10/3a

SUCCESS - saved https://i.ytimg.com/vi/7I8OeQs7cQA/maxresdefault.jpg - as ./data/images/person/25c9ea7730.jpg
SUCCESS - saved https://cdn.shortpixel.ai/client/q_glossy,ret_img,w_777/https://norwaytoday.info/wp-content/uploads/2017/03/Old-woman-777x437.jpg - as ./data/images/person/3ff1f06b9c.jpg
SUCCESS - saved https://basicknowledgecouk.files.wordpress.com/2018/07/pexels-photo-733872.jpeg - as ./data/images/person/16f4e6f4ef.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/a/a0/Andrzej_Person_Kancelaria_Senatu.jpg - as ./data/images/person/deef1c5c0a.jpg
SUCCESS - saved https://imgix.bustle.com/uploads/shutterstock/2019/9/23/c06b1b9c-4524-4411-9814-8f7348bf11d0-shutterstock-1452894716.jpg?w=1020&h=574&fit=crop&crop=faces&auto=format&q=70 - as ./data/images/person/369ce46ec4.jpg
SUCCESS - saved https://images.pexels.com/photos/614810/pexels-photo-614810.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./data/images/person/7a7196265e.jpg
full_body_man
Found: 100 search 

SUCCESS - saved https://www.dhresource.com/0x0/f2/albu/g5/M00/BF/27/rBVaJFkARSeAb4TXAAExlMAKV2U317.jpg - as ./data/images/full_body_man/4368fb34c6.jpg
SUCCESS - saved https://cdn.imgbin.com/24/7/24/imgbin-cute-old-man-standing-on-the-whole-body-man-wearing-gray-collared-shirt-and-gray-pants-EFTwjaX9dMLfADDUfjCs7u5BK.jpg - as ./data/images/full_body_man/7fd5d01b6f.jpg
SUCCESS - saved https://previews.123rf.com/images/tommasolizzul/tommasolizzul1509/tommasolizzul150900030/46100448-muscle-man-bodybuilding-athlete-full-body-portrait.jpg - as ./data/images/full_body_man/031517adee.jpg
SUCCESS - saved https://grid.gograph.com/business-man-full-body-color-black-stock-image_gg82254021.jpg - as ./data/images/full_body_man/c638d66962.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/full-body-picture-smiling-casual-260nw-1175784343.jpg - as ./data/images/full_body_man/c213b4e29b.jpg
SUCCESS - saved https://thumbs.dreamstime.com/z/full-body-man-standing-white-background-32101886.jpg 

SUCCESS - saved https://d2gg9evh47fn9z.cloudfront.net/800px_COLOURBOX6694184.jpg - as ./data/images/full_body_man/4a83fa3eff.jpg
SUCCESS - saved https://previews.123rf.com/images/ruivalesousa/ruivalesousa1001/ruivalesousa100100031/6235443-young-casual-man-full-body-in-a-white-background.jpg - as ./data/images/full_body_man/36ff8f6475.jpg
SUCCESS - saved https://previews.123rf.com/images/newvave/newvave1202/newvave120200100/12684856-casual-young-man-full-body-isolated-on-white.jpg - as ./data/images/full_body_man/14a2d49982.jpg
SUCCESS - saved https://images.assetsdelivery.com/compings_v2/newvave/newvave1202/newvave120200026.jpg - as ./data/images/full_body_man/20053b2cf3.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/man-full-body-business-600w-580599634.jpg - as ./data/images/full_body_man/0c23c049cb.jpg
SUCCESS - saved https://res.cloudinary.com/fleetnation/image/private/c_fit,w_1120/g_south,l_text:style_gothic2:%C2%A9%20Kues,o_20,y_10/g_center,l_watermark4,o_25,y_50/

...

SUCCESS - saved https://st2.depositphotos.com/2299955/11876/i/950/depositphotos_118763690-stock-photo-handsome-man-in-stripes-sitting.jpg - as ./data/images/person_sitting/4865ee2e77.jpg
SUCCESS - saved https://www.vishopper.com/images/products/300x300/PE/12459_man-and-woman-sitting-in-a-cafe-and-talking.jpg - as ./data/images/person_sitting/2da3491716.jpg
SUCCESS - saved https://www.kindpng.com/picc/m/60-604694_back-to-our-people-person-sitting-in-chair.png - as ./data/images/person_sitting/dea3fb13a2.jpg
SUCCESS - saved https://i1.wp.com/www.firstinarchitecture.co.uk/wp-content/uploads/2013/10/Woman-sitting.jpg?fit=707%2C623&ssl=1 - as ./data/images/person_sitting/381412b689.jpg
SUCCESS - saved http://www.dlf.pt/png/big/4/41786_png-person-sitting.png - as ./data/images/person_sitting/8e5452c840.jpg
SUCCESS - saved https://cdn.pixabay.com/photo/2016/01/01/11/00/man-1116396_960_720.png - as ./data/images/person_sitting/fc6e8e84cc.jpg
SUCCESS - saved https://www.pikpng.com/pngl/m/15-156

SUCCESS - saved https://static2.bigstockphoto.com/3/2/3/large1500/323193544.jpg - as ./data/images/person_crouching/606f053ab6.jpg
SUCCESS - saved https://media.istockphoto.com/photos/smiling-young-man-crouching-on-a-white-background-picture-id917899938 - as ./data/images/person_crouching/14b71f6d9a.jpg
SUCCESS - saved https://st4.depositphotos.com/13194036/21676/i/450/depositphotos_216761164-stock-photo-handsome-smiling-young-man-crouching.jpg - as ./data/images/person_crouching/5f0c3c6dbd.jpg
SUCCESS - saved https://images.unsplash.com/photo-1520026278183-f1362e3f5cda?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1000&q=80 - as ./data/images/person_crouching/6c13535ffe.jpg
SUCCESS - saved https://cdn.clipart.email/52a7d754418003c43da4fc5d8835c104_eps-10-vector-illustration-man-sitting-stock-vector-2018-_450-396.jpeg - as ./data/images/person_crouching/c3bf24d6e4.jpg
SUCCESS - saved https://marketplace.canva.com/MAC70U2NJfc/1/screen_2x/canva-man-and-woman%2C-crouchin

SUCCESS - saved https://media.gettyimages.com/photos/full-length-of-man-looking-away-while-crouching-on-grassy-field-sky-picture-id763277561?s=612x612 - as ./data/images/person_crouching/e5d0cb0dfb.jpg
SUCCESS - saved https://i.ytimg.com/vi/8BLvIUUV22U/hqdefault.jpg - as ./data/images/person_crouching/fc6b38eade.jpg
SUCCESS - saved https://c1.wallpaperflare.com/preview/367/508/839/studio-person-man-male.jpg - as ./data/images/person_crouching/633ad3713c.jpg
SUCCESS - saved https://www.cutoutpeople.com/media/cache/full_thumb/th/5c/89/d0/5c89d0e5c279d246222795.jpg - as ./data/images/person_crouching/00832abdc4.jpg
SUCCESS - saved https://st4.depositphotos.com/10313122/25022/i/1600/depositphotos_250229104-stock-photo-full-body-shot-of-young.jpg - as ./data/images/person_crouching/c779bd95a7.jpg
SUCCESS - saved https://image.freepik.com/free-photo/delivery-man-crouching-while-picking-cardboard-box_13339-98346.jpg - as ./data/images/person_crouching/5679bed8c0.jpg
SUCCESS - saved https://th

SUCCESS - saved https://www.sciencesource.com/Doc/TR1_WATERMARKED/2/8/0/c/SS21372089.jpg?d63664448008 - as ./data/images/person_bending/375745b973.jpg
SUCCESS - saved https://media.gettyimages.com/photos/young-man-bending-over-backwards-posing-in-studio-portrait-picture-id200306675-001 - as ./data/images/person_bending/18561a9fa5.jpg
SUCCESS - saved https://images.cdn4.stockunlimited.net/preview1300/silhouette-of-man-bending_1446570.jpg - as ./data/images/person_bending/9e7d1e7f72.jpg
SUCCESS - saved https://png.pngtree.com/png-clipart/20190629/original/pngtree-man-bending-over-and-stretching-hand-pose-element-design-png-image_4084315.jpg - as ./data/images/person_bending/34e877ad5f.jpg
SUCCESS - saved https://www.sciencesource.com/Doc/TR1_WATERMARKED/8/6/c/5/SS21371651.jpg?d63664446599 - as ./data/images/person_bending/39d3d8ca92.jpg
SUCCESS - saved https://media.gettyimages.com/photos/naked-young-man-bending-over-forwards-head-touching-knees-side-view-picture-id200258471-001?s=170667

SUCCESS - saved https://thumbs.dreamstime.com/z/businesswoman-bending-looking-down-against-white-background-96132519.jpg - as ./data/images/person_bending/30c1f1d9da.jpg
SUCCESS - saved https://images.megapixl.com/3198/31984254.jpg - as ./data/images/person_bending/2d38969c10.jpg
SUCCESS - saved https://www.slideteam.net/media/catalog/product/cache/960x720/b/a/back_pain_person_bending_over_and_bending_down_Slide01.jpg - as ./data/images/person_bending/5695d63077.jpg
SUCCESS - saved https://cdn.cnn.com/cnnnext/dam/assets/181022103912-sarcona-before-and-after-exlarge-169.jpg - as ./data/images/person_bending/eef492c175.jpg
SUCCESS - saved https://as1.ftcdn.net/jpg/02/11/01/80/500_F_211018019_jYSiuV8jiDya9s0bp3eNaA2jw5XxtMXT.jpg - as ./data/images/person_bending/98dbd734d8.jpg
SUCCESS - saved https://res.cloudinary.com/fleetnation/image/private/c_fit,w_1120/g_south,l_text:style_gothic2:%C2%A9%20ScienceRF,o_20,y_10/g_center,l_watermark4,o_25,y_50/v1506210047/fcehcjc9hktduyauq94p.jpg - as .

In [35]:
for term in laundry:
    search_and_download(term, driver_path=DRIVER_PATH, number_images=300)

doing_laundry
Found: 100 search results. Extracting links from 0:100
Found: 100 image links, looking for more...
Found: 300 search results. Extracting links from 100:300
Found: 296 image links, looking for more...
Found: 400 search results. Extracting links from 300:400
Found: 300 image links, done!
SUCCESS - saved https://townsquare.media/site/33/files/2011/04/laundry-300x200.jpg?w=980&q=75 - as ./data/images/doing_laundry/7e2ed28382.jpg
SUCCESS - saved https://c8.alamy.com/comp/X8YGFF/man-doing-laundry-at-a-laundromat-X8YGFF.jpg - as ./data/images/doing_laundry/16ca1050ca.jpg
SUCCESS - saved https://www.parentmap.com/sites/default/files/styles/1180x660_scaled_cropped/public/2020-01/young-boy-doing-housework-at-home-picture-id922221748_0.jpg?itok=pgksh_Bg - as ./data/images/doing_laundry/348fe9bf20.jpg
SUCCESS - saved https://previews.123rf.com/images/vadimgozhda/vadimgozhda1412/vadimgozhda141200292/34407350-happy-young-woman-is-doing-laundry-with-washing-machine-at-home-.jpg - as ./d

SUCCESS - saved https://www.eurocheapo.com/blog/wp-content/uploads/2012/05/laundry-Paris.jpg - as ./data/images/doing_laundry/e6adb231b8.jpg
SUCCESS - saved https://as2.ftcdn.net/jpg/01/92/59/51/500_F_192595156_tmq8HDwnvT1hPmKP7RYWxK9U2ntgNoD7.jpg - as ./data/images/doing_laundry/4481ab395a.jpg
SUCCESS - saved https://www.maisondepax.com/wp-content/uploads/2016/08/kids-doing-laundry-2.jpg - as ./data/images/doing_laundry/53722293f6.jpg
SUCCESS - saved https://lionstoothwebsite.files.wordpress.com/2016/03/screen-shot-2016-03-12-at-9-22-22-pm.png?w=656 - as ./data/images/doing_laundry/5d0fac32a7.jpg
SUCCESS - saved https://bloximages.chicago2.vip.townnews.com/bismarcktribune.com/content/tncms/assets/v3/editorial/c/89/c89b912c-5f1e-11e1-816d-001871e3ce6c/4f47e7550d354.image.jpg?resize=400%2C265 - as ./data/images/doing_laundry/36af1cbfb4.jpg
SUCCESS - saved https://image1.masterfile.com/getImage/698-09127542em-teenage-boy-removing-clothes-from-backpack-while-mother-doing-laundry.jpg - as 

/home/mireles/anaconda3/lib/python3.7/site-packages/PIL/Image.py:2817: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)


ERROR - Could not save https://qph.fs.quoracdn.net/main-qimg-35884c6f59fef723c208c17b918fcf87.webp - cannot identify image file <_io.BytesIO object at 0x7faef53143b0>
SUCCESS - saved https://comps.canstockphoto.com/stressed-woman-doing-laundry-at-home-stock-photography_csp70750600.jpg - as ./data/images/doing_laundry/faea3cd8aa.jpg
SUCCESS - saved https://d3dqioy2sca31t.cloudfront.net/Projects/cms/production/000/000/201/medium/65a1b5e61214defd6b34faf665f3d940/437b_WashingInRoom_DB.jpg - as ./data/images/doing_laundry/6e43ad66bc.jpg
SUCCESS - saved https://static1.bigstockphoto.com/1/6/2/large1500/261814570.jpg - as ./data/images/doing_laundry/48636a839a.jpg
SUCCESS - saved https://images.huffingtonpost.com/2016-06-20-1466461373-6806657-giphy-thumb.gif - as ./data/images/doing_laundry/95145df204.jpg
SUCCESS - saved https://cdn7.dissolve.com/p/D25_47_451/D25_47_451_1200.jpg - as ./data/images/doing_laundry/b4e5111ea4.jpg
SUCCESS - saved https://i1.wp.com/offbeathome.com/wp-content/blogs.

SUCCESS - saved https://www.powersmart.ca/img/containers/images/laundry-hero.jpg/f13a36e276efd3b371c121765e20e060.jpg - as ./data/images/doing_laundry/772e76cb98.jpg
SUCCESS - saved https://biohomecares.com/wp-content/uploads/Save-Time-Doing-Laundry.jpg - as ./data/images/doing_laundry/fe8781ccdd.jpg
SUCCESS - saved https://image1.masterfile.com/getImage/698-09075502em-young-man-assisting-female-friend-in-doing-laundry-at-laundromat.jpg - as ./data/images/doing_laundry/587b978f3f.jpg
SUCCESS - saved https://cdn9.dissolve.com/p/D1028_49_497/D1028_49_497_1200.jpg - as ./data/images/doing_laundry/246f3bffac.jpg
SUCCESS - saved https://i.ytimg.com/vi/h7EHeUn3RZs/maxresdefault.jpg - as ./data/images/doing_laundry/00a57aa80c.jpg
SUCCESS - saved https://miro.medium.com/max/1024/1*Vqx9cBj7-rr_pmsXza_WHw.png - as ./data/images/doing_laundry/813f9ea292.jpg
SUCCESS - saved https://image.freepik.com/free-photo/woman-tired-after-doing-laundry-isolated-white_85869-3877.jpg - as ./data/images/doing_l

SUCCESS - saved https://comps.canstockphoto.com/young-husband-man-doing-laundry-at-home-stock-images_csp45555359.jpg - as ./data/images/doing_laundry/f7d923d342.jpg
SUCCESS - saved https://bloximages.chicago2.vip.townnews.com/tucson.com/content/tncms/assets/v3/editorial/0/06/00616358-7a38-503c-b0eb-0907496a1126/5a09c303a7a16.image.jpg?resize=1200%2C800 - as ./data/images/doing_laundry/361a7ecc1e.jpg
SUCCESS - saved https://cdnp.iconscout.com/photo/premium/preview/the-tired-depressed-housewife-doing-laundry-1074053.jpg - as ./data/images/doing_laundry/57158aa209.jpg
SUCCESS - saved https://comps.canstockphoto.com/woman-is-doing-laundry-stock-photography_csp66955906.jpg - as ./data/images/doing_laundry/c0034d4572.jpg
SUCCESS - saved http://www.colonialwalletwisdom.com/wp-content/uploads/2017/11/7-signs-doing-laundry-all-wrong.jpg - as ./data/images/doing_laundry/616f41480b.jpg
SUCCESS - saved https://images.csmonitor.com/csm/2012/01/0119washerdryer.jpg?alias=standard_900x600 - as ./data/

SUCCESS - saved https://www.thespruce.com/thmb/ZQBU2m0Jy5NmCQwmQPIsyGEnlcU=/960x0/filters:no_upscale():max_bytes(150000):strip_icc()/GettyImages-200148269-002-585be6f35f9b586e0223ade0.jpg - as ./data/images/doing_laundry/34bc292551.jpg
SUCCESS - saved https://abc3340.com/resources/media/90972aa2-5c96-44c7-b3cc-f373dbf0e82d-large16x9_531WestMiltonLaundry2.jpg?1559330855419 - as ./data/images/doing_laundry/350d22f877.jpg
SUCCESS - saved https://www.quickanddirtytips.com/sites/default/files/styles/article_main_image/public/images/8890/shutterstock_1085741699-compressor.png?itok=Hc587W8V - as ./data/images/doing_laundry/976c614ed5.jpg
SUCCESS - saved https://cdnp.iconscout.com/photo/premium/preview/young-husband-man-doing-laundry-at-home-2-1050361.jpg - as ./data/images/doing_laundry/05c36da5fe.jpg
ERROR - Could not save http://www.macaronsandmascaraonline.com/wp-content/uploads/2016/10/how-to-do-laundry-in-college.png - cannot identify image file <_io.BytesIO object at 0x7faef531dad0>
SUC

SUCCESS - saved https://www.organicauthority.com/.image/t_share/MTU5MzMwMDg3NDQ0MjkzMjE2/laundry-ccflcr-susandp.jpg - as ./data/images/doing_laundry/b5e918f278.jpg
SUCCESS - saved https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fimages.hellogiggles.com%2Fuploads%2F2016%2F06%2F08074618%2Flaundry.jpg&w=400&c=sc&poi=face&q=85 - as ./data/images/doing_laundry/b8735e04c2.jpg
SUCCESS - saved https://media.istockphoto.com/photos/man-at-home-doing-laundry-picture-id610580964 - as ./data/images/doing_laundry/a3b0c4bc90.jpg
SUCCESS - saved https://cmkt-image-prd.freetls.fastly.net/0.1.0/ps/4772540/910/607/m2/fpnw/wm1/hpepqdkzj9dab6yhz6mhfu7ljch9uggivv1t4rcklkraip1qckun52jjbl7x5r87-.jpg?1531852499&s=f42533b11c18d05c061938542800fdfb - as ./data/images/doing_laundry/0258e20d85.jpg
SUCCESS - saved https://st4.depositphotos.com/1003326/27306/i/1600/depositphotos_273063744-stock-photo-family-doing-laundry.jpg - as ./data/images/doing_laundry/f1bf604b62.jpg
SUCCESS - saved https://cnet4.c